### Principle Rotation Vectors and DCMs
Finding the Principle Rotation Vector (PRV) between two arbitrary coordinate frames can be done in one of two ways:
1. Finding the unit Eigenvalue/Eigenvector of the DCM, or...
2. Directly using some arithmetic formulas (derived from the eigenvector formulation)

In [282]:
import numpy as np
import sympy as sm
from sympy.physics.mechanics import *

def getPRVfromDCM(C):
    foo = (1/2)*(np.trace(C) - 1)
    # foo = 1 corresponds to a 0 deg rotation about any of the three principle axes
    # foo = 0 corresponds to either a 90 or -90 deg rotation about any of the three principle axes
    # foo = -1 corresponds to either a 180 or -180 deg rotation about any of the three principle axes
    u   = np.array([C[1,2]-C[2,1],
                    C[2,0]-C[0,2],
                    C[0,1]-C[1,0]])
    with np.errstate(all='raise'):
        try:
            n = abs(u/np.linalg.norm(u))
        except:
            eigs = np.linalg.eig(C)
            unique, counts = np.unique(eigs.eigenvalues, return_counts=True)
            d = dict(zip(unique,counts))
            if 1.0 in d and d[1.0] == 1:
                idx = np.where(eigs.eigenvalues == 1.0)[0][0]
                n = eigs.eigenvectors[idx]
            else:
                raise ValueError("Error: The QTY of eigenvectors with eigenvalues equal to unity is not exactly 1. Is your rotation matrix Identity or not right-handed?")
            
    Kn = np.array([[0, -n[2], n[1]],
                   [n[2], 0, -n[0]],
                   [-n[1], n[0], 0]])
    
    phis = np.arcsin(-np.trace(np.matmul(Kn,C))/2)
    phic = np.arccos(foo)
    
    if phis >= 0:
        phi = phic
    else:
        phi = -phic

    e = n
    return phi,e

def getPRVfromEuler(a, b, c, rotOrder):
    psi, theta, phi = sm.symbols('psi, theta, phi')
    N = ReferenceFrame('N', indices=('1', '2', '3'))
    B = ReferenceFrame('B', indices=('1', '2', '3'))
    B.orient_body_fixed(N, (psi, theta, phi), rotOrder)
    C = B.dcm(N).subs({psi: np.radians(a),
                       theta: np.radians(b),
                       phi: np.radians(c)})
    return getPRVfromDCM(np.array(C).astype(np.float64))


In [287]:
C = np.array([[0.925417,  0.0296956, 0.377788],
              [0.336824, -0.521281, -0.784102],
              [0.173648,  0.852869, -0.492404]])
display(getPRVfromDCM(C.transpose()))
display(getPRVfromEuler(20,-10,120,'321'))

FB = BN = np.array([[1, 0, 0],
                    [0, 0, 1],
                    [0, -1, 0]])
FN = np.matmul(FB,BN)
display(getPRVfromDCM(FN))

(-2.1461529124034975, array([0.97555042, 0.12165693, 0.18303271]))

(-2.1461528375981986, array([0.97555054, 0.12165573, 0.18303284]))

(3.141592653589793, array([1., 0., 0.]))

In [288]:
foo = np.array([[1, 0, 0],
                [0, 0, 1],
                [0, -1, 0]])
display((1/2)*(foo[0,0] + foo[1,1] + foo[2,2] - 1))
getPRVfromDCM(foo)

0.0

(-1.5707963267948966, array([1., 0., 0.]))

### PRV Addition
PRVs can be added much like DCMs:
1. The DCM method which is $FN(\Phi, \hat{e})=[FB(\Phi_2, \hat{e_2})][BN(\Phi_1, \hat{e_1})]$
2. The direct addition method which is arithmetic (but potentially singular)

In [ ]:
def addPRVs(phi1,e1,phi2,e2):
    sp1 = np.sin(phi1/2)
    sp2 = np.sin(phi2/2)
    cp1 = np.cos(phi1/2)
    cp2 = np.cos(phi2/2)
    phi = 2*np.arccos(cp1*cp2 - sp1*sp2*np.dot(e1,e2))
    e   = (cp2*sp1*e1 + cp1*sp2*e2 + sp1*sp2*np.cross(e1,e2))/np.sin(phi/2)
    return phi,e

def subPRVs(phi,e,phi1,e1):
    sp1  = np.sin(phi1/2)
    sp   = np.sin(phi/2)
    cp1  = np.cos(phi1/2)
    cp   = np.cos(phi/2)
    phi2 = 2*np.arccos(cp*cp1 + sp*sp1*np.dot(e,e1))
    e2   = (cp1*sp*e - cp*sp1*e1 + sp*sp1*np.cross(e,e1))/np.sin(phi2/2)
    return phi2,e2

# def getPRVfromBody(w):
#     np.identity(3) + (1/2)*